In [1]:
import pandas as pd
import numpy as np

kospi200return = pd.read_excel('./data/kospi200return_data.xlsx')
tradeamount_mktcap = pd.read_excel('./data/거래대금_시가총액_코스피200_data.xlsx')
indireturn = pd.read_excel('./data/수익률_코스피200_data.xlsx')
pbr = pd.read_excel('./data/수정PBR_코스피200_data.xlsx')
netbuytradeamount = pd.read_excel('./data/순매수금액_거래대금_코스피200_data.xlsx')
indistd = pd.read_excel('./data/표준편차_코스피200_data.xlsx')
riskfree = pd.read_excel('./data/riskfreerate_data.xlsx')
closingprice = pd.read_excel('./newdata/closingprice.xlsx') 

professionalism = pd.DataFrame()
for i in tradeamount_mktcap.columns:
    if i == 'Date':
        professionalism[i] = tradeamount_mktcap[i]    
    elif '.1'in i:
        pass
    else:
        professionalism[i] = tradeamount_mktcap[i]/tradeamount_mktcap[i+'.1']
        
mktcap = pd.DataFrame()
for i in tradeamount_mktcap.columns:
    if i == 'Date':
        mktcap[i] = tradeamount_mktcap[i]    
    elif '.1'in i:
        mktcap[i.replace('.1','')] = tradeamount_mktcap[i]
    else:
        continue

activity = pd.DataFrame()
for i in netbuytradeamount.columns:
    if i == 'Date':
        activity[i] = netbuytradeamount[i]    
    elif '.1'in i:
        pass
    else:
        activity[i] = netbuytradeamount[i]/netbuytradeamount[i+'.1']

In [2]:
for col in indistd.columns :
    if col == 'Date' :
        continue
    else :
        new_df = pd.DataFrame()
        new_df['Date'] = professionalism['Date']
        new_df['riskfree']=riskfree['Riskfree']
        new_df['kospi200']=kospi200return['KOSPI200']
        new_df['professionalism'] = professionalism[col]
        new_df['pbr'] = pbr[col]
        new_df['activity'] = activity[col]
        new_df['actualreturn'] = indireturn[col]
        new_df['mktcap']= mktcap[col]
        new_df = new_df.dropna()
        new_df.drop(new_df.tail(1).index, inplace=True)
        new_df.to_csv(str(col)+'.csv')

In [3]:
from sklearn.ensemble import RandomForestClassifier
from sklearn.metrics import mean_squared_error
from sklearn.metrics import accuracy_score
from scikitplot.estimators import plot_feature_importances
from sklearn.metrics import precision_score, recall_score, f1_score
from sklearn.model_selection import cross_val_predict
from matplotlib import pyplot as plt
from sklearn.preprocessing import RobustScaler
robscale = RobustScaler()

/Users/jaehongahn/opt/anaconda3/lib/python3.8/site-packages/scipy/__init__.py:138: UserWarning: A NumPy version >=1.16.5 and <1.23.0 is required for this version of SciPy (detected version 1.23.1)
  warnings.warn(f"A NumPy version >={np_minversion} and <{np_maxversion} is required for this version of "


In [4]:
from scipy.optimize import minimize

In [5]:
import statsmodels.api as sm

In [6]:
start_date = '2020-07-31'
end_date =  '2022-07-31'
dates = pd.date_range(start_date, end_date, freq='M')
investable_lm = pd.DataFrame(index=dates)
prediction = pd.DataFrame(index=dates)



for col in indistd.columns:
    if col == 'Date':
        continue
    else:
        new_df = pd.DataFrame()
        new_df = pd.read_csv(str(col)+'.csv')
        del new_df['Unnamed: 0']
        new_df.dropna()
    #up: 0 / down: 1    
    new_df['Direction'] = np.where(new_df['actualreturn'].values > new_df['riskfree'].values , 0, 1)
    new_df = new_df.loc['2011-12-31'<=new_df.Date]
    new_df=new_df.replace([np.inf, -np.inf],0.0)
    new_df.dropna()
    if len(new_df)==128:
        pass
    else: 
        continue
    
    prof_ = robscale.fit_transform(new_df['professionalism'].values.reshape(-1,1))
    pbr_ = robscale.fit_transform(new_df['pbr'].values.reshape(-1,1))
    act_ = robscale.fit_transform(new_df['activity'].values.reshape(-1,1))
    kos2_ = robscale.fit_transform(new_df['kospi200'].values.reshape(-1,1))
    rf_ = robscale.fit_transform(new_df['riskfree'].values.reshape(-1,1))
    new_df.insert(1, 'prof_scaled',prof_)
    new_df.insert(1, 'pbr_scaled',pbr_)
    new_df.insert(1, 'act_scaled',act_)
    new_df.insert(1, 'kos2_scaled',kos2_)
    new_df.insert(1, 'rf_scaled',rf_)
    new_df['mktrt'] = new_df['kospi200']-new_df['riskfree']
    train = new_df.loc[new_df.Date <= '2018-04-30']
    valid = new_df.loc[(new_df.Date > '2018-04-30') & (new_df.Date<= '2020-06-30')]
    test = new_df.loc[new_df.Date >'2020-06-30']

    if len(train)==77 and len(valid)==26 and len(test)==25:
        pass
    else:
        continue

    if train.empty:
        continue
    if valid.empty:
        continue
    if test.empty:
        continue
    
    
    
    X_train = train.loc[:,['prof_scaled','pbr_scaled','act_scaled','kos2_scaled','rf_scaled']]
    y_train = train.Direction
    
    X_train_ = train.loc[:,['professionalism','pbr','activity','mktrt','mktcap']]
    y_train_ = train['actualreturn']-train['riskfree'] 

    X_valid = valid.loc[:,['prof_scaled','pbr_scaled','act_scaled','kos2_scaled','rf_scaled']]
    y_valid = valid.Direction
    
    X_valid_ = valid.loc[:,['professionalism','pbr','activity','mktrt','mktcap']]
    y_valid_ = valid['actualreturn']-valid['riskfree']  
    
    X_test = test.loc[:,['prof_scaled','pbr_scaled','act_scaled','kos2_scaled','rf_scaled']]
    y_test = test.Direction

    X_test_ = test.loc[:,['professionalism','pbr','activity','mktrt','mktcap']]
    y_test_ = test['actualreturn']-test['riskfree'] 

    multiLm = sm.OLS(y_train_, sm.add_constant(X_train_)).fit()

    if multiLm.rsquared_adj >= 0.6 and multiLm.f_pvalue < 0.05:
        pass
    else:
        continue
    
    lm_pred = multiLm.predict(sm.add_constant(X_test_))
    lm_pred=lm_pred.to_numpy()
    prediction[col] = lm_pred
    
    model = RandomForestClassifier(random_state=42, n_estimators=100)
    model.fit(X_train, y_train)
    
    y_pred = model.predict(X_test)
    investable_lm[col] = y_pred

In [8]:
from datetime import datetime, time

Set_lm = {}
for d in investable_lm.index:
    stocks = []
    for stock in investable_lm.columns:
        if investable_lm[stock][d] == 0:
            stocks.append(stock)
    Set_lm[datetime.strftime(d, '%Y-%m-%d')] = stocks

In [9]:
start_date = '2020-07-31'
end_date = '2022-07-31'
columns = investable_lm.columns
dates = pd.date_range(start_date, end_date, freq='M')
invSet_lm = pd.DataFrame(index=dates, columns=columns)

In [10]:
for date,stocks in Set_lm.items():
    for stock in stocks:
        invSet_lm[stock][date] = prediction[stock][date]

In [12]:
indiv=indireturn[:][144:]
indiv=indiv.dropna(axis=1)
temp=indiv[:][0:103]
temp.set_index('Date',inplace=True)

In [13]:
investableSet_lm = pd.concat([temp, invSet_lm], axis=0)

In [14]:
rf=riskfree[144:272]
rf.set_index('Date', inplace=True)

In [15]:
Riskfree=riskfree[247:272]
Riskfree.set_index('Date', inplace=True)

In [16]:
def MeanVarianceOptimization(expected, cov_matrix, riskfree):
    num = len(expected)
    def objfuc(weights):
        weights = np.array(weights)
        sigma=np.dot(weights.T, np.dot(cov_matrix, weights))
        rtn = np.sum(expected*weights)
        objfuction = (rtn-riskfree)/np.sqrt(sigma)
        return -objfuction 
    constraints=({"type":"eq","fun":lambda w: np.sum(w)-1},
                {"type":"ineq","fun":lambda w: w})
    bounds = tuple((0,1) for x in range(num))
    init = [1/num for x in range(num)]
    result = minimize(objfuc, init, method="SLSQP", bounds=bounds, constraints=constraints)
    return result

In [17]:
Return_lm = pd.DataFrame(index=invSet_lm.index, columns=['Return'])
count1=103
count2=0
weights=[]
tickers=[]
each_return=[]

for date in invSet_lm.index:
    temp1 = temp[:][count1:count1+1]
    temp = indiv[:][:]
    temp.set_index('Date', inplace=True)
    temp = temp[:][0:count1]
    invSet1_lm = invSet_lm[:][count2:count2+1]
    invSet2_lm = invSet_lm[:][count2:count2+1]
    actual = pd.concat([temp1, invSet2_lm], axis=0)
    actual = actual.dropna(axis=1)
    actual = actual.iloc[:1,:]
    prediction_lm = pd.DataFrame(index=invSet_lm.index)
    for col in actual.columns:
        prediction_lm[col] = prediction[col]

    
    invSet1_lm=invSet1_lm.dropna(axis=1)
    result = MeanVarianceOptimization(invSet1_lm.mean(),prediction_lm.cov(),rf.loc[date])
    weights.append(result.x)
    tickers.append(invSet1_lm.columns)
    each_return.append(actual.to_numpy()*result.x)
    Return_lm['Return'][datetime.strftime(date, '%Y-%m-%d')]=np.sum(actual.to_numpy()*result.x)
    count1=count1+1
    count2=count2+1


In [18]:
stockprice=closingprice[247:272]
stockprice=stockprice.rename(columns={'Symbol':'Date'})
stockprice.set_index('Date', inplace=True)

In [20]:
initial_amount = 100000000
balance=initial_amount
gain=0
remain_amount=0
period_return=1
for i in range(len(Return_lm.index)):
#     print('Date: ',Return_lm.index[i])
#     print('Balance: ', 'KRW',initial_amount)
    for j in range(len(tickers[i])):
        stockweights='{:.2f}'.format(weights[i][j])
        stockreturns='{:.2f}'.format(each_return[i][0][j])
        if float(stockweights) == 0.00:
            pass
        else:
#             print('Ticker:',tickers[i][j].strip('A'), 'Weight:',stockweights)
#             print('Long '+ tickers[i][j].strip('A') + ':', 'KRW',float(stockweights)*initial_amount, '수량:', int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i]), '현재가', int(stockprice[tickers[i][j]][i]))
#             print('Each Return: ', stockreturns, '%')

            gain = gain + int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])*float(stockreturns)/100
            purchased = int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])
            remain_amount = remain_amount + purchased

#     print()
    remain_amount = initial_amount - remain_amount
#     print('Net Gain: ', 'KRW',gain, '잔여잔고: ', 'KRW', remain_amount)
#     print('Profit (%): ', gain/initial_amount*100, '%')
    term_profit = gain/initial_amount
    period_return = period_return*(1+term_profit)
#     print()
#     print()
#     print('잔고',initial_amount, gain, remain_amount)                                                                                                                        
    initial_amount = initial_amount + gain
#     print('새 seed: ',initial_amount)
    gain=0
    remain_amount=0
# print('Final Balance: ', initial_amount, 'Gain: ', (initial_amount-balance), 'Annualized return: ',(period_return**(1/len(Return_lm.index))-1)*100, '%')




In [73]:
def robo_advisor(amount):
    initial_amount = float(amount)
    balance=initial_amount
    gain=0
    remain_amount=0
    period_return=1
    listforstd=[]
    for i in range(len(Return_lm.index)):
        print('Date: ',Return_lm.index[i])
        print('Balance: ', 'KRW',initial_amount)
        for j in range(len(tickers[i])):
            stockweights='{:.2f}'.format(weights[i][j])
            stockreturns='{:.2f}'.format(each_return[i][0][j])
            if float(stockweights) == 0.00:
                pass
            else:
                print('Ticker:',tickers[i][j].strip('A'), 'Weight:',stockweights)
                print('Long '+ tickers[i][j].strip('A') + ':', 'KRW',float(stockweights)*initial_amount, '수량:', int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i]), '현재가', int(stockprice[tickers[i][j]][i]))
                print('Each Return: ', stockreturns, '%')

                gain = gain + int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])*float(stockreturns)/100
                purchased = int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])
                remain_amount = remain_amount + purchased

        print()
        remain_amount = initial_amount - remain_amount
        print('Net Gain: ', 'KRW',gain)
        print('Profit (%): ', gain/initial_amount*100, '%')
        term_profit = gain/initial_amount
        period_return = period_return*(1+term_profit)
        listforstd.append(term_profit)
        initial_amount = initial_amount + gain
        gain=0
        remain_amount=0
    print('Final Balance: KRW', '{:.2f}'.format(initial_amount), 'Gain: KRW', '{:.2f}'.format(initial_amount-balance), 'Average monthly return: ','{:.2f}'.format((period_return**(1/len(Return_lm.index))-1)*100), '%', 'Risk: ','{:.2f}'.format(np.std(listforstd)*100),'%')




In [74]:
amount_invest = input('투입 금액을 입력해주세요: ')
robo_advisor(amount_invest)

투입 금액을 입력해주세요:  100000000


Date:  2020-07-31 00:00:00
Balance:  KRW 100000000.0
Ticker: 010060 Weight: 0.15
Long 010060: KRW 15000000.0 수량: 267 현재가 56100
Each Return:  5.76 %
Ticker: 042670 Weight: 0.10
Long 042670: KRW 10000000.0 수량: 1406 현재가 7110
Each Return:  4.80 %
Ticker: 005690 Weight: 0.36
Long 005690: KRW 36000000.0 수량: 1589 현재가 22650
Each Return:  18.64 %
Ticker: 007810 Weight: 0.06
Long 007810: KRW 6000000.0 수량: 370 현재가 16200
Each Return:  2.40 %
Ticker: 010770 Weight: 0.32
Long 010770: KRW 32000000.0 수량: 10702 현재가 2990
Each Return:  8.14 %

Net Gain:  KRW 10799880.212000001
Profit (%):  10.799880212000001 %
Date:  2020-08-31 00:00:00
Balance:  KRW 110799880.212
Ticker: 010060 Weight: 0.20
Long 010060: KRW 22159976.042400002 수량: 342 현재가 64700
Each Return:  4.80 %
Ticker: 069620 Weight: 0.04
Long 069620: KRW 4431995.20848 수량: 38 현재가 116000
Each Return:  1.19 %
Ticker: 019170 Weight: 0.01
Long 019170: KRW 1107998.80212 수량: 8 현재가 136500
Each Return:  -0.16 %
Ticker: 042670 Weight: 0.45
Long 042670: KRW 49

In [57]:
initial_amount = 100000000
balance=initial_amount
gain=0
remain_amount=0
period_return=1

stock_monthly = {}

for i in range(len(Return_lm.index)):
#     print('Date: ',Return_lm.index[i])
#     print('Balance: ', 'KRW',initial_amount)
    list_temp = []

    for j in range(len(tickers[i])):
        stockweights='{:.2f}'.format(weights[i][j])
        stockreturns='{:.2f}'.format(each_return[i][0][j])
        if float(stockweights) == 0.00:
            pass
        else:
#             print('Ticker:',tickers[i][j].strip('A'), 'Weight:',stockweights)
#             print('Long '+ tickers[i][j].strip('A') + ':', 'KRW',float(stockweights)*initial_amount, '수량:', int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i]), '현재가', int(stockprice[tickers[i][j]][i]))
#             print('Each Return: ', stockreturns, '%')
            
            gain = gain + int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])*float(stockreturns)/100
            purchased = int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])
            remain_amount = remain_amount + purchased
            list_temp.append([tickers[i][j],int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])])
#     print()
    stock_monthly[Return_lm.index[i]]=list_temp
    remain_amount = initial_amount - remain_amount
#     print('Net Gain: ', 'KRW',gain, '잔여잔고: ', 'KRW', remain_amount)
#     print('Profit (%): ', gain/initial_amount*100, '%')
    term_profit = gain/initial_amount
    period_return = period_return*(1+term_profit)
#     print()
#     print()
#     print('잔고',initial_amount, gain, remain_amount)                                                                                                                        
    initial_amount = initial_amount + gain
#     print('새 seed: ',initial_amount)
    gain=0
    remain_amount=0
# print('Final Balance: ', initial_amount, 'Gain: ', (initial_amount-balance), 'Annualized return: ',(period_return**(1/len(Return_lm.index))-1)*100, '%')
file=pd.DataFrame.from_dict(stock_monthly, orient='index')
file.to_csv(str(balance)+'result.csv')

In [104]:
def robo_advisor_excel(amount):
    initial_amount = float(amount)
    balance=initial_amount
    gain=0
    remain_amount=0
    period_return=1
    listforstd=[]
    stock_monthly = {}
    portfolio_result = {}
    
    for i in range(len(Return_lm.index)):
        list_temp=[]
        list_result=[]
        for j in range(len(tickers[i])):
            stockweights='{:.2f}'.format(weights[i][j])
            stockreturns='{:.2f}'.format(each_return[i][0][j])
            if float(stockweights) == 0.00:
                pass
            else:
                gain = gain + int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])*float(stockreturns)/100
                purchased = int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])*int(stockprice[tickers[i][j]][i])
                remain_amount = remain_amount + purchased
                list_temp.append([tickers[i][j],int(float(stockweights)*initial_amount/stockprice[tickers[i][j]][i])])
        stock_monthly[Return_lm.index[i]]=list_temp
        remain_amount = initial_amount - remain_amount
        term_profit = gain/initial_amount
        period_return = period_return*(1+term_profit)
        listforstd.append(term_profit)
        initial_amount = initial_amount + gain
        gain=0
        remain_amount=0
    portfolio_result['예상 자산']='{:.2f}'.format(initial_amount)
    portfolio_result['예상 수익률']='{:.2f}'.format((period_return**(1/len(Return_lm.index))-1)*100)
    portfolio_result['예상 위험']='{:.2f}'.format(np.std(listforstd)*100)
    
    file=pd.DataFrame.from_dict(stock_monthly, orient='index')
    file.to_csv('input_'+str(amount)+'_stock.csv')
    file2=pd.DataFrame.from_dict(portfolio_result, orient='index')
    file2.to_csv('input_'+str(amount)+'_result.csv')
    


In [105]:
input_amount = input('투입 금액을 입력해주세요: ')
robo_advisor_excel(input_amount)

투입 금액을 입력해주세요:  5000000
